In [1]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
With SwiftPM flags: []
Working in: /tmp/tmp_gh0cyox/swift-install
/home/martin/swift/usr/bin/swift-build: /home/martin/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/martin/swift/usr/lib/swift/linux/libFoundation.so)
Fetching https://github.com/mxcl/Path.swift
Completed resolution in 2.99s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
/home/martin/swift/usr/bin/swiftc: /home/martin/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/martin/swift/usr/bin/swiftc)
Compile Swift Module 'Path' (9 sources)
/home/martin/swift/usr/bin/swiftc: /home/martin/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/martin/swift/usr/bin/swiftc)

/home/martin/swift/usr/bin/swift: /home/martin/anaconda3/envs/swift/lib/libuuid.so.1: no vers

In [2]:
import Path
import Foundation

# A Swift Tour - Playground 2 Notebook

In [3]:
var path = Path.cwd/"GuidedTour.playground/contents.xcplayground"

In [4]:
var results : [String] = []

# Parsing the XML

Apparently, the xcode playground is defined in an xml format that looks sth like this:
```
<?xml version='1.0' encoding='UTF-8' standalone='yes'?>
<playground version='6.0' target-platform='ios' display-mode='rendered'>
   <pages>
        <page name='Simple Values'/>
        <page name='Control Flow'/>
        <page name='Functions and Closures'/>
        <page name='Objects and Classes'/>
        <page name='Enumerations and Structures'/>
        <page name='Protocols and Extensions'/>
        <page name='Error Handling'/>
        <page name='Generics'/>
        <page name='License'/>
    </pages>
</playground>

```
So, we need to parse it! In Swift we can achieve this by creating an XMLParserDelegate to collect the page names.


In [5]:
class MyParser: XMLParserDelegate {
    
    var pages: [String] = []
    
    func parser(_ parser: XMLParser, didStartElement elementName: String, namespaceURI: String?, qualifiedName qName: String?, attributes attributeDict: [String : String]) {
        if elementName == "page" {
            pages.append(attributeDict["name"]!)
        }
    }
}

In [6]:
let delegate = MyParser()

In [7]:
let parser = XMLParser(contentsOf: path.url)
parser?.delegate = delegate
parser?.parse()
delegate.pages

▿ 9 elements
  - 0 : "Simple Values"
  - 1 : "Control Flow"
  - 2 : "Functions and Closures"
  - 3 : "Objects and Classes"
  - 4 : "Enumerations and Structures"
  - 5 : "Protocols and Extensions"
  - 6 : "Error Handling"
  - 7 : "Generics"
  - 8 : "License"


Great! Now we have the pages. Let's grab them

In [8]:
func adaptToNotebook(section: Dictionary<String, String>) -> Dictionary<String, Any> {
    var newSection: Dictionary<String, Any> = [:]
    for (k, v) in section {
        newSection[k] = v
    }
    let emptyMeta: Dictionary<String,String> = [:]
    newSection["metadata"] = emptyMeta
    if (section["cell_type"]=="code") {
        newSection["outputs"] = Array<String>()
        newSection["execution_count"] = 0
    }
    return newSection
}

In [9]:
func parseLines(lines: [String]) -> Array<Dictionary<String, String>> {
    var sections: Array<Dictionary<String,String>> = []
    var curSection: Dictionary<String,String> = [:]
    var ctxt: String = ""
    func shouldStartNewSection(ctxt: String, line: String) -> Bool {
        if (ctxt == "") {
            return true
        }
        if (ctxt == "markdown" && !line.starts(with: "//:")) {
            return true
        }
        if (ctxt == "code" && line.starts(with: "//:")) {
            return true
        }
        return false;
    }
    func getLineContents(ctxt: String, line: String) -> String {
        return ctxt == "markdown" ? line.substring(from: 3) : line
    }
    for line in lines.filter({!$0.isEmpty}) {
        if shouldStartNewSection(ctxt: ctxt , line: line) {
            if (!curSection.isEmpty) {
                sections.append(curSection)
            }
            ctxt = line.starts(with: "//:") ? "markdown" : "code";
            curSection = [
                "cell_type": ctxt,
                "source": ""
            ]
        }
        curSection["source"]? += getLineContents(ctxt: ctxt, line: line) + "\n"
    }
    if (!curSection.isEmpty) {
        sections.append(curSection)
    }
    return sections
}

In [10]:
for (i ,page) in delegate.pages.enumerated() {
    let pagename = page + ".xcplaygroundpage"
    let fp = Path.cwd/"GuidedTour.playground"/"Pages"/pagename/"Contents.swift" 
    let pageContents = String.init(contentsOfFile: fp.description)
    let lines = pageContents.components(separatedBy: "\n")
    let sections = parseLines(lines: lines)
    let adaptedSections = sections.map(adaptToNotebook(sectio)
    let notebook : Dictionary<String, Any> = ["cells": adaptedSections,
                  "metadata": [
                      "kernelspec": [
                       "display_name": "Swift",
                       "language": "swift",
                       "name": "swift"
                      ],
                      "language_info": [
                       "file_extension": ".swift",
                       "mimetype": "text/x-swift",
                       "name": "swift",
                       "version": ""
                      ]
                    ],
                 "nbformat": 4,
                 "nbformat_minor": 2
    ]    
    JSONSerialization.data(withJSONObject: notebook).write(to: (Path.cwd / "notebooks"/(String(i) + " - " + page+".ipynb")))
}

: 